![](./usd-ois.png)

In [24]:
import numpy as np

def val(sig, l1):
    # Note Features
    S0 = 41.76                 # Initial stock price ($)
    K = 25.06                  # Coupon barrier and downside threshold ($)
    coupon_rate = 0.1025       # Contingent coupon rate per annum
    coupon_payment = 25.625    # Contingent coupon per observation date ($)
    principal = 1000           # Principal amount per note ($)
    share_delivery_amount = 23.9464  # Shares delivered if below threshold at maturity
    call_start_step = 2        # Issuer can call starting from the second observation date

    # Market Parameters
    r1 = 0.037242092 # Risk-free interest rate (2%)
    r2 = 0.037098736
    q = 0.04842                  # Dividend yield (3%)
    sigma = sig              # Stock volatility (25%)

    # Time Parameters
    T = 2.0                    # Total time to maturity (2 years)
    N = 8                     # Number of time steps (quarterly observations)
    dt = T / N                 # Time step size

    # Binomial Tree Parameters
    u = np.exp(sigma * np.sqrt(dt))        # Up factor
    d = np.exp(-sigma * np.sqrt(dt))       # Down factor
    p = (np.exp((r1 - q) * dt) - d) / (u - d)  # Risk-neutral probability

    # Check for arbitrage
    if not (0 < p < 1):
        raise ValueError("Risk-neutral probability is not between 0 and 1. Check parameters.")

    # Build Stock Price Tree
    stock_tree = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

    # Initialize Payoff Matrix
    value_tree = np.zeros_like(stock_tree)

    # Determine Payoffs at Maturity
    for j in range(N + 1):
        S_T = stock_tree[j, N]
        if S_T >= K:
            # Receive principal back
            value_tree[j, N] = principal + (coupon_payment if S_T >= K else 0)
        else:
            # Receive share delivery amount
            value_tree[j, N] = share_delivery_amount * S_T

    # Backward Induction
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            S_t = stock_tree[j, i]
            # Calculate expected continuation value
            continuation_value = np.exp(-r2 * dt) * (p * value_tree[j, i + 1] + (1 - p) * value_tree[j + 1, i + 1])
            
            # Add contingent coupon if barrier is breached
            if S_t >= K:
                continuation_value += coupon_payment
            
            # Check if issuer will call the note
            if i >= call_start_step:
                # Calculate call payoff
                call_payoff = principal + (coupon_payment if S_t >= K else 0)
                # Issuer will call if it's optimal
                node_value = min(call_payoff, continuation_value)
            else:
                node_value = continuation_value
            
            value_tree[j, i] = node_value

    # Output the Note's Present Value
    note_value = value_tree[0, 0]
    l1.append(note_value)
    print(f"The estimated fair value of the note is: ${note_value:.2f}")

In [25]:
sig = [0.37779, 0.36529, 0.34846, 0.33187, 0.31841, 0.30770, 0.29896, 0.29334, 0.28662]

In [26]:
l1 = []
for i in sig:
    val(i, l1)

The estimated fair value of the note is: $964.12
The estimated fair value of the note is: $967.68
The estimated fair value of the note is: $972.47
The estimated fair value of the note is: $990.76
The estimated fair value of the note is: $994.49
The estimated fair value of the note is: $997.46
The estimated fair value of the note is: $999.89
The estimated fair value of the note is: $1001.44
The estimated fair value of the note is: $1003.31


In [27]:
l1

[964.1173849424858,
 967.6795239820171,
 972.4709589524332,
 990.7606836194402,
 994.4942125263381,
 997.463773396379,
 999.8863714469996,
 1001.4438120189466,
 1003.3057516654233]

In [29]:
np.mean(l1)

987.9580525056068

In [ ]:
import numpy as np

In [17]:
dt = (2*365+2)/365
dt

2.0054794520547947

In [14]:
r1 = np.log(0.927865)/dt

In [15]:
r1

-0.03717985889038573